In [4]:
import prosperity3bt
import pandas as pd
import itertools
import numpy as np
from tqdm import tqdm

from prosperity3bt.runner import run_backtest
from prosperity3bt.models import TradeMatchingMode
from prosperity3bt.file_reader import FileSystemReader
from pathlib import Path

from importlib import reload

In [5]:
def day_profit(result):
    last_timestamp = result.activity_logs[-1].timestamp

    profit = 0
    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        profit += row.columns[-1]

    return profit

def backtest_days(trader, days=[-2, -1, 0], data_path='.', round_num=1):
    profits = []
    for day in days:
        backtest_results = prosperity3bt.runner.run_backtest(trader, file_reader=FileSystemReader(Path(data_path)), round_num=round_num, day_num=day, print_output=False, trade_matching_mode=TradeMatchingMode.all, no_names=True, show_progress_bar=False)
        profit = day_profit(backtest_results)
        profits.append(profit)
    return profits

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [16]:
import r5_base_model_finale

reload(r5_base_model_finale)

# PARAMS = {
#     Product.RESIN: {
#         "fair_value": 10000,
#         "take_width": 1,
#         "clear_width": 0,
#         # for making
#         "disregard_edge": 1,  # disregards orders for joining or pennying within this value from fair
#         "join_edge": 2,  # joins orders within this edge
#         "default_edge": 4,
#         "soft_position_limit": 10,
#     },
#     Product.KELP: {
#         "take_width": 1,
#         "clear_width": 0,
#         "prevent_adverse": True,
#         "adverse_volume": 15,
#         "reversion_beta": 0,
#         "disregard_edge": 1,
#         "join_edge": 0,
#         "default_edge": 1,
#     },
#     Product.INK: {
#         'ink_change_threshold_pct': 0.015,
#         'ink_window_size': 25,
#         'ink_position_limit': 50,
#         'clear_price_thresh': 4
#     },

#     'ink_lookback_window': 20,            # Number of periods for momentum lookback
#     'ink_momentum_threshold': 0.005,      # Change in return required to trigger a trade
#     'ink_past_lag': 1,                     # Number of periods to look back for past return
#     'ink_clear_threshold': 0.0001,         # Threshold below which we clear the position,


#     Product.VOLC_ROCK: {
#         'rock_change_threshold_pct': 0.015,
#         'rock_window_size': 25,
#         'rock_position_limit': 50,
#         'clear_price_thresh': 4
#     },

#     # individual picnic legs – we wnat tiny passive edges
#     Product.CROISSANTS: {"edge": 5},
#     Product.JAMS:       {"edge": 20},
#     Product.DJEMBES:    {"edge": 20},

#     # basket mis‑pricing thresholds, max for 1
#     Product.PB1: {"spread_threshold": 52, "clear_thresh": 5, "max_qty_per_leg": 38},
#     Product.PB2: {"spread_threshold": 130,  "max_qty_per_leg": 100},


#     "10500_threshold": 0.25,
#     "10500_max_qty": 20,
#     "10250_threshold": 0.25,
#     "10250_max_qty": 50,
#     "10000_threshold": 0.25,
#     "10000_max_qty": 50,
#     "9750_threshold": 1,
#     "9750_max_qty": 50,
#     "9500_threshold": 1.75,
#     "9500_max_qty": 1,

#     "r3_day": 4,

#     "rock_z_score_threshold": 3.7,
#     "rock_z_score_take_profit": 0.1
# }

class Product:
    RESIN = "RAINFOREST_RESIN"
    KELP = "KELP"
    INK = "SQUID_INK"

    CROISSANTS = "CROISSANTS"
    JAMS       = "JAMS"
    DJEMBES    = "DJEMBES"

    PB1 = "PICNIC_BASKET1"
    PB2 = "PICNIC_BASKET2"
    MAC = "MAGNIFICENT_MACARONS"

    VOLC_ROCK                = "VOLCANIC_ROCK"
    VOUCHER_9500             = "VOLCANIC_ROCK_VOUCHER_9500"
    VOUCHER_9750             = "VOLCANIC_ROCK_VOUCHER_9750"
    VOUCHER_10000            = "VOLCANIC_ROCK_VOUCHER_10000"
    VOUCHER_10250            = "VOLCANIC_ROCK_VOUCHER_10250"
    VOUCHER_10500            = "VOLCANIC_ROCK_VOUCHER_10500"


PARAMS = {
    Product.RESIN: {
        "fair_value": 10000,
        "take_width": 1,
        "clear_width": 0,
        # for making
        "disregard_edge": 1,  # disregards orders for joining or pennying within this value from fair
        "join_edge": 2,  # joins orders within this edge
        "default_edge": 4,
        "soft_position_limit": 10,
    },
    Product.KELP: {
        "take_width": 1,
        "clear_width": 0,
        "prevent_adverse": True,
        "adverse_volume": 15,
        "reversion_beta": 0,
        "disregard_edge": 1,
        "join_edge": 0,
        "default_edge": 1,
    },
    Product.INK: {
        'ink_change_threshold_pct': 0.015,
        'ink_window_size': 25,
        'ink_position_limit': 50,
        'clear_price_thresh': 4
    },

    'ink_lookback_window': 20,            # Number of periods for momentum lookback
    'ink_momentum_threshold': 0.005,      # Change in return required to trigger a trade
    'ink_past_lag': 1,                     # Number of periods to look back for past return
    'ink_clear_threshold': 0.0001,         # Threshold below which we clear the position,


    Product.VOLC_ROCK: {
        'rock_change_threshold_pct': 0.015,
        'rock_window_size': 25,
        'rock_position_limit': 50,
        'clear_price_thresh': 4
    },

    # individual picnic legs – we wnat tiny passive edges
    Product.CROISSANTS: {"edge": 5},
    Product.JAMS:       {"edge": 20},
    Product.DJEMBES:    {"edge": 20},

    # basket mis‑pricing thresholds, max for 1
    Product.PB1: {"spread_threshold": 52, "clear_thresh": 5, "max_qty_per_leg": 38},
    Product.PB2: {"spread_threshold": 130,  "max_qty_per_leg": 100},


    "10500_threshold": 0.25,
    "10500_max_qty": 20,
    "10250_threshold": 0.25,
    "10250_max_qty": 50,
    "10000_threshold": 0.25,
    "10000_max_qty": 50,
    "9750_threshold": 1,
    "9750_max_qty": 50,
    "9500_threshold": 1.75,
    "9500_max_qty": 1,

    "r3_day": 4,

    # "rock_z_score_threshold": 3.7,
    # "rock_z_score_take_profit": 0.1
}


param_grid = {
    # "rock_z_score_threshold": [1.1, 1.2, 1.3, 1.4, 3.7, 3.8, 3.9, 4.0],
    "rock_z_score_threshold": [1.7, 1.8, 1.9, 2, 2.1],
    # "rock_z_score_take_profit": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    "rock_z_score_take_profit": [0.01, 0.02, 0.03, 0.04, 0.05],
}


param_profits = []

for params in tqdm(generate_param_combinations(param_grid)):
    trader = r5_base_model_finale.Trader(params.update(PARAMS))
    days = [2, 3, 4]
    profits = backtest_days(trader, days=days, data_path='.', round_num=5)
    for d in days:
        params[f'profit_day_{d}'] = profits[days.index(d)]
    param_profits.append(params)

100%|██████████| 25/25 [04:56<00:00, 11.87s/it]


In [17]:
df = pd.DataFrame(param_profits)
df['tot_profit'] = df[[f'profit_day_{d}' for d in days]].sum(axis=1)
df.sort_values(by='tot_profit', ascending=False, inplace=True)
df

,rock_z_score_threshold,rock_z_score_take_profit,RAINFOREST_RESIN,KELP,SQUID_INK,ink_lookback_window,ink_momentum_threshold,ink_past_lag,ink_clear_threshold,VOLCANIC_ROCK,...,10000_max_qty,9750_threshold,9750_max_qty,9500_threshold,9500_max_qty,r3_day,profit_day_2,profit_day_3,profit_day_4,tot_profit
0,1.7,0.01,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
13,1.9,0.04,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
23,2.1,0.04,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
22,2.1,0.03,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
21,2.1,0.02,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
20,2.1,0.01,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
19,2.0,0.05,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
18,2.0,0.04,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
17,2.0,0.03,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
16,2.0,0.02,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0


In [13]:
df.sort_values(by='profit_day_3', ascending=False)

,rock_z_score_threshold,rock_z_score_take_profit,RAINFOREST_RESIN,KELP,SQUID_INK,ink_lookback_window,ink_momentum_threshold,ink_past_lag,ink_clear_threshold,VOLCANIC_ROCK,...,10000_max_qty,9750_threshold,9750_max_qty,9500_threshold,9500_max_qty,r3_day,profit_day_2,profit_day_3,profit_day_4,tot_profit
0,1,0.2,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
11,3,0.4,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
23,5,0.8,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
2,1,0.6,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
3,1,0.8,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
4,1,1.0,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
5,2,0.2,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
6,2,0.4,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
7,2,0.6,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0
8,2,0.8,"{'fair_value': 10000, 'take_width': 1, 'clear_...","{'take_width': 1, 'clear_width': 0, 'prevent_a...","{'ink_change_threshold_pct': 0.015, 'ink_windo...",20,0.005,1,0.0001,"{'rock_change_threshold_pct': 0.015, 'rock_win...",...,50,1,50,1.75,1,4,44030.0,-3597.0,25853.0,66286.0


In [12]:
profits

[0.0]

In [ ]:
df.to_csv('backtests/inktrader_bt_results.csv')

,ink_change_threshold_pct,ink_window_size,profit_day_-2,profit_day_-1,profit_day_0,tot_profit
0,0.02,5,4273.0,3450.0,0.0,7723.0
1,0.02,10,4121.0,3450.0,0.0,7571.0
2,0.02,15,4121.0,3450.0,0.0,7571.0
3,0.03,5,0.0,3450.0,0.0,3450.0
4,0.03,10,0.0,3450.0,0.0,3450.0
5,0.03,15,0.0,3450.0,0.0,3450.0
6,0.04,5,0.0,3450.0,0.0,3450.0
7,0.04,10,0.0,3450.0,0.0,3450.0
8,0.04,15,0.0,3450.0,0.0,3450.0
